In [21]:
# General Imports
import glob
import shutil

# Setup Spark Session
from pyspark.sql import SparkSession

# Get Spark Functions Needed
from pyspark.sql.functions import col, udf, split, explode, lit

# Get Datatypes needed for DataFrame manipulation
from pyspark.sql.types import IntegerType, StringType, StructType, StructField, StringType

# Setup Spark Session
sc = SparkSession \
        .builder \
        .master("local[*]") \
        .appName("data_clean_up") \
        .getOrCreate()

# Print Spark Version being run
print("Spark V: ", sc.version)

Spark V:  3.3.2


In [22]:
# *********************************
# *** English Data Preparations ***
# *********************************

# Load English CSV File into a Dataframe
df_english = sc.read.csv("data/english.csv", header=True, inferSchema=True)
print(f'Count (raw): {df_english.count()}')

# Print Columns
print(f'English Source Columns: {df_english.columns}\n')

# Filter only lavbels that we mathc with the German counter parts
filter_values = ["love", "hate", "neutral", "anger", "happiness", "surprise", "sadness", "worry", "enthusiasm"]
df_english_filtered = df_english.filter(col("sentiment").isin(filter_values))

# Remove unnecessary column
df_english_filtered = df_english_filtered.drop("tweet_id")

# Rename Columns
df_english_filtered = df_english_filtered.withColumnRenamed("sentiment", "emotion_en")
df_english_filtered = df_english_filtered.withColumnRenamed("content", "sentence_en")

# Make sure the column order to the same for both german and english csv files
df_english_filtered = df_english_filtered.select('sentence_en', 'emotion_en')

print(f'Count (filtered): {df_english_filtered.count()}')

# Group By for Details & Count
df_english_grouped = df_english_filtered.groupBy('emotion_en').count()

# Show Groupings and Respetive Counts
print('\nGrouped & Count by "emotion"')
df_english_grouped.show(truncate=0)


# Save Dataframe to CSV
directory_path = 'data/spark_data_parts'
df_english_filtered.coalesce(1).write.csv(directory_path, header=True, mode="overwrite")

file_pattern = 'part-00000*.csv'
file_path = glob.glob(directory_path + '/' + file_pattern)[0]

shutil.move(file_path, './data/data_en.csv')

Count (raw): 40000
English Source Columns: ['tweet_id', 'sentiment', 'content']

Count (filtered): 35692

Grouped & Count by "emotion"
+----------+-----+
|emotion_en|count|
+----------+-----+
|love      |3842 |
|hate      |1323 |
|neutral   |8638 |
|anger     |110  |
|happiness |5209 |
|surprise  |2187 |
|sadness   |5165 |
|worry     |8459 |
|enthusiasm|759  |
+----------+-----+



'./data/data_en.csv'

In [23]:
# ********************************
# *** German Data Preparations ***
# ********************************

# Load German JSON File into a Dataframe
df_german = sc.read.json("data/german.json")
print(f'Count (raw): {df_german.count()}')

# Print Columns
print(f'German Source Columns: {df_german.columns}\n')


# We Need to "split" the "artice_emotion" column since the ETH team listed
# multiple emotions in one column
# In order to "explode" the column to it's distinct rows
df_german_exploded = df_german.select('*', explode('article_emotion').alias('emotion_de'))

# Rename Column
df_german_exploded = df_german_exploded.withColumnRenamed("title", "sentence_de")

# Remove unnecessary column
df_german_exploded = df_german_exploded.drop("article_id")
df_german_exploded = df_german_exploded.drop("article_stance")
df_german_exploded = df_german_exploded.drop("paragraphs")
df_german_exploded = df_german_exploded.drop("source")
df_german_exploded = df_german_exploded.drop("article_emotion")
df_german_exploded = df_german_exploded.drop("snippet")

# Make sure the column order to the same for both german and english csv files
df_german_exploded = df_german_exploded.select('sentence_de', 'emotion_de')

print(f'Count (filtered): {df_german_exploded.count()}')

# Group By for Details & Count
df_german_grouped = df_german_exploded.groupBy('emotion_de').count()

# Show Groupings and Respetive Counts
print('\nGrouped & Count by "emotion"')
df_german_grouped.show(truncate=0)


# Save Dataframe to CSV
directory_path = 'data/spark_data_parts'
df_german_exploded.coalesce(1).write.csv(directory_path, header=True, mode="overwrite")

file_pattern = 'part-00000*.csv'
file_path = glob.glob(directory_path + '/' + file_pattern)[0]

shutil.move(file_path, './data/data_de.csv')


Count (raw): 1970
German Source Columns: ['article_emotion', 'article_id', 'article_stance', 'paragraphs', 'snippet', 'source', 'title']

Count (filtered): 2568

Grouped & Count by "emotion"
+------------+-----+
|emotion_de  |count|
+------------+-----+
|Vertrauen   |316  |
|Freude      |140  |
|Ärger       |226  |
|Überraschung|369  |
|Traurigkeit |184  |
|Antizipation|774  |
|Unklar      |314  |
|Angst       |154  |
|Ekel        |29   |
|Keine       |62   |
+------------+-----+



'./data/data_de.csv'

In [26]:
# **************************************
# *** English <-> German Emotion Key ***
# **************************************


from pyspark.sql.functions import col, udf, split, explode, lit
from pyspark.sql.types import IntegerType, StringType, StructType, StructField, StringType

# Emotion Dictionary English <-> German
# This key/value setup was previsoulsy created for linking purposes
emotion_key = {
    "boredom"    : "---",
    "love"       : "Vertrauen",
    "relief"     : "---",
    "fun"        : "---",
    "hate"       : "Ekel",
    "neutral"    : "Unklar",
    "anger"      : "Ärger",
    "happiness"  : "Freude",
    "surprise"   : "Überraschung", 
    "sadness"    : "Traurigkeit", 
    "worry"      : "Angst",
    "enthusiasm" : "Antizipation", 
    "empty "     : "---",
    "---"        : "Keine",
}

# Create the schema for the DataFrame
schema = StructType([
    StructField("emotion_en", StringType(), nullable=False),
    StructField("emotion_de", StringType(), nullable=False)
])

# Convert the dictionary to a list of tuples
data = [(key, value) for key, value in emotion_key.items()]

# Create the PySaprk DataFrame
df_emotion_key = sc.createDataFrame(data, schema)

df_emotion_key.show()

# Extend the English and German onto the datasets
df_german_exploded = df_german_exploded.join(df_emotion_key, on="emotion_de", how="left")
df_english_filtered = df_english_filtered.join(df_emotion_key, on="emotion_en", how="left")

# New sentence column
df_german_exploded = df_german_exploded.withColumn("sentence_en", lit(None))
df_english_filtered = df_english_filtered.withColumn("sentence_de", lit(None))

df_german_exploded.show()
df_english_filtered.show()


# Using Deep Translator to proxy to Google Translate
from deep_translator import GoogleTranslator


# Define Tranlation Functions
def translate_en_to_de(row):
    translated = GoogleTranslator(source='en', target='de').translate(row["sentence_en"])
    row["sentence_de"] = translated

def translate_de_to_en(row):
    translated = GoogleTranslator(source='de', target='en').translate(row["sentence_de"])
    row["sentence_en"] = translated
    

df_german_exploded.foreach(translate_de_to_en)  


+----------+------------+
|emotion_en|  emotion_de|
+----------+------------+
|   boredom|         ---|
|      love|   Vertrauen|
|    relief|         ---|
|       fun|         ---|
|      hate|        Ekel|
|   neutral|      Unklar|
|     anger|       Ärger|
| happiness|      Freude|
|  surprise|Überraschung|
|   sadness| Traurigkeit|
|     worry|       Angst|
|enthusiasm|Antizipation|
|    empty |         ---|
|       ---|       Keine|
+----------+------------+

+------------+--------------------+----------+----------+-----------+----------+
|  emotion_de|         sentence_de|emotion_en|emotion_en|sentence_en|emotion_en|
+------------+--------------------+----------+----------+-----------+----------+
|   Vertrauen|Adoptiert zu sein...|      love|      love|       null|      love|
|      Freude|Österreichs Verfa...| happiness| happiness|       null| happiness|
|      Freude|Visana gewinnt er...| happiness| happiness|       null| happiness|
|       Ärger|Masern: Erste Kin...|     anger

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 260.0 failed 1 times, most recent failure: Lost task 0.0 in stage 260.0 (TID 659) (richards-mbp.localdomain executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/homebrew/Cellar/apache-spark/3.3.2/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/opt/homebrew/Cellar/apache-spark/3.3.2/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/opt/homebrew/Cellar/apache-spark/3.3.2/libexec/python/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/homebrew/Cellar/apache-spark/3.3.2/libexec/python/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iter